# Attencion Mask

Vamos criar um modelo capaz de prever sequências de comprimento igual a 10 tokens.

Um modelo Transformer consiste em várias partes principais:
* 1 - Camada de Embedding: Teransforma as palavras em vetores numéricos de tamanho fixo.
* 2 - Mecanismo de Atenção: Permite o modelo foque em diferentes partes da entrada.
* 3 - Camadas Encoder e Decoder: Processam os dados sequencialmente.
* 4 - Camada Linear e Softmax: Para predições finais.

Para este projeto o grande objetivo é implementar o item 2, mas para deixar o exemplo funcional, implementarei também os itens 1 e 4

### Imports iniciais

In [25]:
# Imports
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

### Camada Embedding

A função embedding é utilizada para converter entradas sequenciais em vetores densos de tamanho fixo. Esses vetores são conhecidos como embeddings e são uma parte fundamental, em especial dos modelos de PLN. 

Esses embeddings são fundamentais para modelos de aprendizado profundo em PLN, pois fornecem uma representação rica e densa de palavras ou tokens, capturando informações contextuais e semânticas que são essenciais para tarefas como tradução automática, classificação de texto, entre outras.

In [7]:
# Define a função para criar uma matriz de embedding
def embedding(input, vocab_size, dim_model):

    # Cria a matriz de embedding onde cada linha representa um token do vocabulário
    # A matriz é inicializada com valores aleatórios normalmente distribuídos
    embed = np.random.randn(vocab_size, dim_model)

    # Para cada índice de token no input, seleciona o embedding correspondente da matriz
    # Retorna um array de embeddings correspondentes à sequência de entrada
    return np.array([embed[i] for i in input])

### Mecanismo de Atenção

No Transformer, Q, K e V são derivados da mesma entrada em camadas de atenção do encoder, mas de entradas diferentes no decoder (Q vem da saída da camada anterior do decoder, enquanto K e V vêm da saída do encoder). O mecanismo de atenção calcula um conjunto de pontuações (usando o produto escalar entre Q e K, daí o nome "scaled dot-product attention"), aplica uma função softmax para obter pesos de atenção e usa esses pesos para ponderar os values, criando uma saída que é uma combinação ponderada das informações relevantes de entrada.

Este processo permite que o modelo dê "atenção" às partes mais relevantes da entrada para cada parte da saída, o que é especialmente útil em tarefas como tradução, onde a relevância de diferentes palavras da entrada pode variar dependendo da parte da frase que está sendo traduzida.

### Função de Ativação Softmax

A função softmax é uma função de ativação amplamente utilizada em redes neurais, especialmente em cenários de classificação, onde é importante transformar valores brutos de saída (logits) em probabilidades que somam 1. Abaixo, está o código da função softmax com comentários em cada linha explicando seu funcionamento:

In [8]:
# Função de ativação softmax
def softmax(x):
    
    # Calcula o exponencial de cada elemento do input, ajustado pelo máximo valor no input 
    # para evitar overflow numérico
    e_x = np.exp(x - np.max(x))
    
    # Divide cada exponencial pelo somatório dos exponenciais ao longo do último eixo (axis=-1)
    # O reshape(-1, 1) garante que a divisão seja realizada corretamente em um contexto multidimensional
    return e_x / e_x.sum(axis=-1).reshape(-1, 1)

### Scale Dot Product

A função scaled_dot_product_attention() é um componente do mecanismo de atenção em modelos Transformer. Ela calcula a atenção entre conjuntos de queries (Q), keys (K) e values (V). 

Essencialmente, essa função permite que o modelo dê importância diferenciada a diferentes partes da entrada, um aspecto chave que torna os modelos Transformer particularmente eficazes para tarefas de PLN e outras tarefas sequenciais.

In [9]:
# Define a função para calcular a atenção escalada por produto escalar
def scaled_dot_product_attention(Q, K, V):
    
    # Calcula o produto escalar entre Q e a transposta de K
    matmul_qk = np.dot(Q, K.T)
    
    # Obtém a dimensão dos vetores de chave
    depth = K.shape[-1]
    
    # Escala os logits dividindo-os pela raiz quadrada da profundidade
    logits = matmul_qk / np.sqrt(depth)
    
    # Aplica a função softmax para obter os pesos de atenção
    attention_weights = softmax(logits)
    
    # Multiplica os pesos de atenção pelos valores V para obter a saída final
    output = np.dot(attention_weights, V)
    
    # Retorna a saída ponderada
    return output

### Saída do Modelo com Operação Linear e Softmax

A função linear_and_softmax() é uma combinação de uma camada linear seguida por uma função softmax, comumente usada em modelos de aprendizado profundo, especialmente em tarefas de classificação. 

In [10]:
# Define a função que aplica uma transformação linear seguida de softmax
def linear_and_softmax(input):
    
    # Inicializa uma matriz de pesos com valores aleatórios normalmente distribuídos
    # Esta matriz conecta cada dimensão do modelo (dim_model) a cada palavra do vocabulário (vocab_size)
    weights = np.random.randn(dim_model, vocab_size)
    
    # Realiza a operação linear (produto escalar) entre a entrada e a matriz de pesos
    # O resultado, logits, é um vetor que representa a entrada transformada em um espaço de maior dimensão
    logits = np.dot(input, weights)
    
    # Aplica a função softmax aos logits
    # Isso transforma os logits em um vetor de probabilidades, onde cada elemento soma 1
    return softmax(logits)

### Construindo o Modelo Final

In [11]:
# Função do modelo final
def transformer_model(input):
    
    # Embedding
    embedded_input = embedding(input, vocab_size, dim_model)

    # Mecanismo de Atenção 
    attention_output = scaled_dot_product_attention(embedded_input, embedded_input, embedded_input)
    
    # Camada linear e softmax
    output_probabilities = linear_and_softmax(attention_output)

    # Escolhendo os índices com maior probabilidade
    output_indices = np.argmax(output_probabilities, axis=-1)
    
    return output_indices

---
### Hiperparâmetros iniciais

In [48]:
# Dimensão do modelo
dim_model = 4

# Comprimento da sequência
seq_length = 5

# Tamanho do vocabulário
vocab_size = 100

---

## Usando o Modelo Para as Previsões

In [12]:
# Gerando dados aleatórios para a entrada do modelo
input_sequence = np.random.randint(0, vocab_size, seq_length)

In [62]:
print("Sequência de Entrada:", input_sequence)

Sequência de Entrada: [69 88 29 87 67]


In [65]:
# Fazendo previsões com o modelo
output = transformer_model(input_sequence)

In [66]:
print("Saída do Modelo:", output)

Saída do Modelo: [29 45 45 29 70]


---

### Passo a passo da execução

In [61]:
# Gerando dados aleatórios para a entrada do modelo
input_sequence = np.random.randint(0, vocab_size, seq_length)
input_sequence

array([69, 88, 29, 87, 67])

In [51]:
# Embedding
embedded_input = embedding(input_sequence, vocab_size, dim_model)
embedded_input[0:1]

array([[-1.2168222 ,  1.30473414,  0.84480715, -1.35817327]])

In [52]:
# Mecanismo de Atenção 
attention_output = scaled_dot_product_attention(embedded_input, embedded_input, embedded_input)
attention_output

array([[-0.86992256,  1.10027359,  1.00464841, -1.08533723],
       [-0.92628395,  0.25245825, -0.65779795,  0.30722767],
       [-0.18069875,  0.56752686, -0.21765976,  0.17738769],
       [-0.60207574,  1.08489145,  1.1428699 , -0.95905918],
       [ 1.59459658,  1.25569574,  2.27023003, -0.06384452]])

In [58]:
# Camada linear e softmax
output_probabilities = linear_and_softmax(attention_output)
output_probabilities

array([[1.81159921e-03, 5.97001201e-03, 1.69792926e-03, 2.81770553e-04,
        2.79675701e-02, 1.35030199e-03, 6.04385815e-03, 1.28588590e-02,
        5.74703887e-03, 5.08303414e-03, 1.85165348e-01, 7.22589511e-02,
        2.20139299e-03, 1.85332595e-03, 1.69767761e-03, 6.68713649e-03,
        7.47125021e-04, 1.11479663e-03, 3.34936017e-05, 2.60155492e-03,
        8.95294771e-03, 4.23934888e-04, 1.81900494e-03, 1.16625039e-02,
        5.44774381e-03, 1.91159052e-03, 5.64731639e-03, 3.35465872e-04,
        4.89316961e-02, 1.20067010e-03, 7.45868378e-04, 1.58248462e-03,
        8.74595059e-04, 3.11512037e-03, 1.68693105e-02, 2.25706713e-03,
        3.04682834e-04, 9.67517185e-06, 2.89540808e-04, 8.13229494e-04,
        5.52225768e-04, 3.87057710e-03, 1.96601758e-04, 1.02986887e-02,
        7.06578368e-02, 2.73706206e-02, 3.77780301e-03, 1.54753011e-03,
        1.07286219e-02, 1.16311981e-03, 2.10905909e-04, 6.53756345e-04,
        3.02549000e-03, 4.21814010e-02, 1.53654872e-02, 3.920053

In [60]:
# Escolhendo os índices com maior probabilidade
output_indices = np.argmax(output_probabilities, axis=-1)
output_indices

array([10, 83,  5, 10,  1], dtype=int64)